# Imports and Global Variables

In [2]:
import torch
import geoopt
import numpy as np

from scipy.linalg import expm

from logalike import Logalike

In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


$$\sigma_i = \{ 0, 1, \ldots, M_\sigma, D \}$$

$\sigma_i$ = state at target $i$ at site $\sigma$ consisting of an unedited, mutated, or deleted base. More specifically:

$0$ = unedited

$1 \ldots M_\sigma $ = mutation that are feasible at site $\sigma$

$D$ = deleted

edit = mutated $\cup$ deleted

In [3]:
# comment all these lines out if have actual data. This is just simulated ata
slow_guides = ['AGCTGCTTAGGGCGCAGCCT', 'CTCCTTGCGTTGACCCGCTT', 'TATTGCCTCTTAATCGTCTT']
medium_guides = ['AATCCCTAGTAGATTAGCCT', 'CACAGAACTTTATGACGATA', 'TTAAGTTTGAGCTCGCGCAA']
fast_guides = ['TAATTCCGGACGAAATCTTG', 'CTTCCACGGCTCTAGTACAT', 'CCTCCCGTAGTGTTGAGTCA']
cassette_sites = slow_guides+medium_guides
lineage, Q = simulate_lineage(cassette_sites=cassette_sites, num_init_cells=2, init_death_prob=0.1,
                     init_repr_prob=0.75, cancer_prob=1e-3, tree_depth=10)

Q = torch.tensor(Q) # TODO: make this Q compact

from anthony.conversion_utils import networkx_to_ete, get_ete_cm
etetree = networkx_to_ete(lineage) # comment this out if have actual data
cm = get_ete_cm(etetree)
cm = torch.tensor(cm.to_numpy()) # TODO: ensure not loosing dimensionality when converting from PD Dataframe --> NP array --> Torch tensor

In [5]:
# num mutations at each site
num_mutations = torch.tensor([4, 4, 4, 4, 4, 4])
rho = torch.tensor(2, dtype=torch.float64)

# initial guess for points
num_cells = cm.shape[0]
manifold = geoopt.Lorentz(k=rho)
points = manifold.random_normal(num_cells, dtype=torch.float64)

In [6]:
l = Logalike(rho=rho,
             character_matrix=cm,
             init_points=points,
             num_mutations=num_mutations,
             S=6,)

opt = geoopt.optim.RiemannianAdam(l.parameters(), lr=1e-3)
for i in range(num_cells):
    opt.zero_grad()
    loss = l.forward(Q, i)
    loss.backward()
    opt.step()
    

tensor(2029)
0.18160257605909189 	 0.16666666666666666 1.0438464716396523 1.0438464716396523
0.18160257605909183 	 0.16666666666666666 1.043846471639652 1.043846471639652
0.0 	 0.16666666666666666 -0.00019076124272200071 0.0
0.16666666690926574 	 0.16666666666666666 1.0000000000000002 1.0000000000000002
0.20479993820387982 	 0.16666666666666666 1.1085123496034128 1.1085123496034128
0.0 	 0.16666666666666666 0.0 0.0
tensor(2029)
0.18483357020884192 	 0.16666666666666666 1.053091364152727 1.053091364152727
0.18483357020884192 	 0.16666666666666666 1.053091364152727 1.053091364152727
0.0 	 0.16666666666666666 -0.00023098265891984636 0.0
0.1666666670223533 	 0.16666666666666666 1.0000000000000002 1.0000000000000002
0.21365656688014661 	 0.16666666666666666 1.132227627856201 1.132227627856201
0.0 	 0.16666666666666666 0.0 0.0
tensor(1678)
0.00044599872984755584 	 0.16666666666666666 0.051729994965061955 0.051729994965061955
0.0004459987298475533 	 0.16666666666666666 0.05172999496506181 0.0

AssertionError: 